# Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow. 

Let's get our vectorDB from before.

## Vectorstore retrieval


In [1]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
# !pip install lark

### Similarity Search

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = './resources/docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [5]:
print(vectordb._collection.count())

209


In [6]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [7]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [8]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [9]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [10]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

### Addressing Diversity: Maximum marginal relevance

Last class we introduced one problem: how to enforce diversity in the search results.
 
`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [11]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [12]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [13]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

Note the difference in results with `MMR`.

In [14]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [15]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [16]:
docs_mmr[1].page_content[:100]

'into his office and he said, "Oh, professo r, professor, thank you so much for your \nmachine learnin'

### Addressing Specificity: working with metadata

In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [17]:
question = "what did they say about regression in the third lecture?"

In [18]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"./resources/docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)

In [19]:
for d in docs:
    print(d.metadata)

{'source': './resources/docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 0}
{'source': './resources/docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 14}
{'source': './resources/docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 4}


### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:
 
1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [20]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [21]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

**Note:** The default model for `OpenAI` ("from langchain.llms import OpenAI") is `text-davinci-003`. Due to the deprication of OpenAI's model `text-davinci-003` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [22]:
document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [23]:
question = "what did they say about regression in the third lecture?"

**You will receive a warning** about predict_and_parse being deprecated the first time you executing the next line. This can be safely ignored.

In [24]:
docs = retriever.get_relevant_documents(question)

In [25]:
for d in docs:
    print(d.metadata)

### Additional tricks: compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text. 

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. 

In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [27]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [28]:
# Wrap our vectorstore
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [29]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [30]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data."
- "there's also a software package called Octave that you can download for free off the Internet."
- "it has somewhat fewer features than MATLAB, but it's free, and for the purposes of this class, it will work for just about everything."
- "once a colleague of mine at a different university, not at Stanford, actually teaches another machine learning course."
----------------------------------------------------------------------------------------------------
Document 2:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write 

## Combining various techniques

In [31]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [32]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data."
- "there's also a software package called Octave that you can download for free off the Internet."
- "it has somewhat fewer features than MATLAB, but it's free, and for the purposes of this class, it will work for just about everything."
- "once a colleague of mine at a different university, not at Stanford, actually teaches another machine learning course."
----------------------------------------------------------------------------------------------------
Document 2:

"Oh, it was the MATLAB."
----------------------------------------------------------------------------------------------------
Document 3:

- "All the homeworks can be done in MATLAB or Octave"
- "The program 

## Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents. 

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [33]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [34]:
# Load PDF
loader = PyPDFLoader("./resources/docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [35]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [36]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes")

In [37]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

## Experiment on your own

In [70]:
# Set up example Chroma DB

machine_learning_texts = [
    "Machine Learning, a branch of artificial intelligence, focuses on the development of algorithms that enable computers to learn from and make predictions or decisions based on data. Unlike traditional programming, where instructions are explicitly given, ML algorithms improve automatically through experience.",
    "Supervised learning, a primary method within Machine Learning, involves training an algorithm on a labeled dataset, where the correct output is known, enabling the model to learn over time to predict the output when given new data.",
    "Unsupervised learning, another core technique in Machine Learning, deals with input data without labeled responses. The system tries to learn the patterns and the structure from such data by itself, often used for clustering or anomaly detection.",
    "Reinforcement learning is a type of Machine Learning where an algorithm learns to perform an action from experience by making decisions in a game-like scenario. It optimizes its decision-making process based on the rewards received for actions taken.",
    "Deep Learning, a subset of Machine Learning, utilizes neural networks with many layers. These algorithms are particularly powerful in handling vast amounts of data and are behind many advances in image and speech recognition, language translation, and autonomous vehicles.",
    "Feature engineering is a crucial step in the Machine Learning process, involving the selection, modification, and creation of input variables or features that are used to train the model. It significantly impacts the performance of ML algorithms.",
    "Overfitting in Machine Learning occurs when a model learns the detail and noise in the training data to the extent that it performs poorly on new data. Techniques such as cross-validation, regularization, and pruning are used to avoid this issue.",
    "Machine Learning models require large volumes of data for training. However, the quality and relevance of the data are equally important, as biased or incomplete data can lead to inaccurate models that perpetuate biases or fail to perform as intended.",
    "The ethical implications of Machine Learning are a growing concern, as the technology can inadvertently reinforce existing biases, infringe on privacy, or be used in manipulative ways. Ensuring fairness, accountability, and transparency in ML models is crucial.",
    "The future of Machine Learning promises advancements in personalized medicine, environmental protection, and automated systems. However, it also poses challenges in job displacement, security, and ensuring ethical use of the technology.",
]

machine_learning_metadatas = [
    {"summary": "Introduction to ML", "category": "Basics"},
    {"summary": "Supervised Learning Overview", "category": "Learning Types"},
    {"summary": "Unsupervised Learning Explained", "category": "Learning Types"},
    {"summary": "Basics of Reinforcement Learning", "category": "Learning Types"},
    {"summary": "Deep Learning and Neural Networks", "category": "Advanced Topics"},
    {"summary": "Feature Engineering Techniques", "category": "Model Preparation"},
    {"summary": "Handling Overfitting in Models", "category": "Model Evaluation"},
    {"summary": "Importance of Data Quality", "category": "Data Preparation"},
    {"summary": "Ethical Considerations in ML", "category": "Ethics and Future"},
    {"summary": "Future Trends in Machine Learning", "category": "Ethics and Future"}
]


ml_db = Chroma.from_texts(
    texts=machine_learning_texts,
    metadatas=machine_learning_metadatas,
    embedding=embedding)

In [71]:
# Vanilla Similarity Search

question = "What are some types of Machine Learning algorithms?"
docs_ss = ml_db.similarity_search(question,k=3)

print(docs_ss[0].page_content[:100])
print(docs_ss[1].page_content[:100])
print(docs_ss[2].page_content[:100])

Machine Learning, a branch of artificial intelligence, focuses on the development of algorithms that
Deep Learning, a subset of Machine Learning, utilizes neural networks with many layers. These algori
Supervised learning, a primary method within Machine Learning, involves training an algorithm on a l


In [72]:
# MMR

docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

print(docs_mmr[0].page_content[:100])
print(docs_mmr[1].page_content[:100])
print(docs_mmr[2].page_content[:100])

So I just want to start by showing you a f un video. Remember at the last lecture, the 
initial lect
And let's see. Oh, and the goal of the projec t should really be for you to do a publishable 
piece 
over the last 15, 20 years in an electronic format.  
Turns out that most of you probably use learni


In [73]:
# Filter by Metadata

metadata_filer_docs = ml_db.similarity_search(
    question,
    k=3,
    filter={"category": "Learning Types"}
)

metadata_filer_docs

[Document(page_content='Supervised learning, a primary method within Machine Learning, involves training an algorithm on a labeled dataset, where the correct output is known, enabling the model to learn over time to predict the output when given new data.', metadata={'summary': 'Supervised Learning Overview', 'category': 'Learning Types'}),
 Document(page_content='Reinforcement learning is a type of Machine Learning where an algorithm learns to perform an action from experience by making decisions in a game-like scenario. It optimizes its decision-making process based on the rewards received for actions taken.', metadata={'summary': 'Basics of Reinforcement Learning', 'category': 'Learning Types'}),
 Document(page_content='Unsupervised learning, another core technique in Machine Learning, deals with input data without labeled responses. The system tries to learn the patterns and the structure from such data by itself, often used for clustering or anomaly detection.', metadata={'summary

In [88]:
# Self-query retriever

metadata_field_info = [
    AttributeInfo(
        name="summary",
        description="A summary of the text data",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The category associated the ML text data",
        type="string",
    ),
]

document_content_description = "ML notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    ml_db,
    document_content_description,
    metadata_field_info,
    verbose=True
)

docs = retriever.get_relevant_documents(question)

print(docs)

for d in docs:
    print(d.metadata)

[Document(page_content='Machine Learning, a branch of artificial intelligence, focuses on the development of algorithms that enable computers to learn from and make predictions or decisions based on data. Unlike traditional programming, where instructions are explicitly given, ML algorithms improve automatically through experience.', metadata={'summary': 'Introduction to ML', 'category': 'Basics'}), Document(page_content='Deep Learning, a subset of Machine Learning, utilizes neural networks with many layers. These algorithms are particularly powerful in handling vast amounts of data and are behind many advances in image and speech recognition, language translation, and autonomous vehicles.', metadata={'summary': 'Deep Learning and Neural Networks', 'category': 'Advanced Topics'}), Document(page_content='Machine Learning models require large volumes of data for training. However, the quality and relevance of the data are equally important, as biased or incomplete data can lead to inaccu

In [89]:
# Contextual Compression

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ml_db.as_retriever()
)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- Machine Learning
- branch of artificial intelligence
- development of algorithms
- enable computers to learn from and make predictions or decisions based on data
- traditional programming
- instructions are explicitly given
- ML algorithms improve automatically through experience
----------------------------------------------------------------------------------------------------
Document 2:

- Deep Learning
- Machine Learning
- Neural networks
- Many layers
- Powerful in handling vast amounts of data
- Advances in image and speech recognition, language translation, and autonomous vehicles
----------------------------------------------------------------------------------------------------
Document 3:

Supervised learning, Machine Learning, labeled dataset, correct output, model, learn, predict, new data.
----------------------------------------------------------------------------------------------------
Document 4:

- Reinforcement learning
- Type of Machine Learning


In [90]:
# Combine Various Techniques (MMR, Filter by Metadata, Contextual Compression)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ml_db.as_retriever(search_type = "mmr"),
    filter={"category": "Learning Types"}
)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Number of requested results 20 is greater than number of elements in index 10, updating n_results = 10
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings

Document 1:

- Machine Learning
- branch of artificial intelligence
- development of algorithms
- enable computers to learn from and make predictions or decisions based on data
- traditional programming
- instructions are explicitly given
- ML algorithms improve automatically through experience
----------------------------------------------------------------------------------------------------
Document 2:

Unsupervised learning, another core technique in Machine Learning, deals with input data without labeled responses. The system tries to learn the patterns and the structure from such data by itself, often used for clustering or anomaly detection.
----------------------------------------------------------------------------------------------------
Document 3:

cross-validation, regularization, and pruning


In [91]:
# Traditional Retreival (SVM Retreiver)
svm_retriever = SVMRetriever.from_texts(machine_learning_texts,embedding)
docs_svm=svm_retriever.get_relevant_documents(question)
print(docs_svm[0])

page_content='Reinforcement learning is a type of Machine Learning where an algorithm learns to perform an action from experience by making decisions in a game-like scenario. It optimizes its decision-making process based on the rewards received for actions taken.'


/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [92]:
# Traditional Retreival (TF-IDF Retriever)
tfidf_retriever = TFIDFRetriever.from_texts(splits)
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
print(docs_tfidf[0])

page_content="there's also a smaller minority of students th at will sometimes try to prove — develop the \ntheory of machine learning further or try to  prove theorems about machine learning. So \nthey're usually great projects of all of those types with applications and machine learning \nbeing the most common. Anything else? Okay, cool.  \nSo that was it for logistics. Let's talk about  learning algorithms. So can I have the laptop \ndisplay, please, or the projector? Actually, co uld you lower the big sc reen? Cool. This is \namazing customer service. Thank you. I see. Okay, cool. Okay. No, that's fine. I see. \nOkay. That's cool. Thanks. Okay.  \nBig screen isn't working toda y, but I hope you can read things  on the smaller screens out \nthere. Actually, [inaudible] I think this room just got a new projector that — someone \nsent you an excited email — was it just on Frid ay? — saying we just got a new projector \nand they said 4,000-to-1 something or othe r brightness ratio. I d